In [ ]:
!pip3 install autoawq

In [ ]:
!cd /workspace/repos && git clone https://github.com/arcee-ai/mergekit.git && cd mergekit && pip install -e .

In [ ]:
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM

model_a_name = "Qwen/QwQ-32B"
model_b_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"

model_a = AutoModelForCausalLM.from_pretrained(model_a_name)
tokenizer_a = AutoTokenizer.from_pretrained(model_a_name)
del model_a, tokenizer_a
gc.collect()

model_b = AutoModelForCausalLM.from_pretrained(model_b_name)
tokenizer_b = AutoTokenizer.from_pretrained(model_b_name)
del model_b, tokenizer_b
gc.collect()

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/8.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
!mergekit-yaml /workspace/repos/agentic-ai/tutorials/merge_models.yml /workspace/models

In [ ]:
from awq import AutoAWQForCausalLM

model = AutoAWQForCausalLM.from_pretrained(
    model_path, **{"low_cpu_mem_usage": True, "use_cache": False}
)

quant_config={
  "zero_point": True,
  "q_group_size": 128,
  "w_bit": 4,
  "version": "GEMM"
}

In [ ]:
!echo $HF_HOME